In [1]:
import numpy as np
from time import time
import torch
import matplotlib.pyplot as plt
import seaborn as sns
%run Rule.ipynb
%run Algorithm_1.ipynb
%run Model.ipynb
from tqdm.notebook import tqdm
import seaborn as sns

In [318]:
def general_2d_solver(visc, u0, W, mu = None, sigma = None, T = 1, X = 1, dt = None):
    
    # visc - viscosity in front of Laplacian     
    # u0 - initial condition
    # xi - space time noise
    # mu - drift term of the equation
    # sigma - diffusion term of the equation
    # T - time horizon
    # X - space domain is [0,X]
    
    # space time grid
    M, N =  W.shape[1], W.shape[2]
    if dt is None: dt = T/M
    
    # Time deirvative of the noise
    dW = np.zeros(W.shape)
    dW[:,1:,:] = np.diff(W, axis = 1)
    
    lap = - np.array([[2*N**2*np.cos(2*np.pi*k1/(N*X)) + 2*N**2*np.cos(2*np.pi*k2/(N*X)) for k2 in range(N)] for k1 in range(N)]) + 2*N**2 + 2*N**2 
    lap[0,0] = 1
    lap_ = 0.5*dt*visc*lap
    
    mu_ = np.vectorize(mu) if mu is not None else 1
    sigma_ = np.vectorize(sigma) if sigma is not None else 1
    
    soln = np.zeros(dW.shape)
    soln[:,0,:,:] = u0 # set the initial condition
    
    w = np.fft.fft2(u0, axes = [-2,-1])
    
    for i in tqdm(range(1, M)):
        
        # non-linearities in the physical space
        RHS_drift = mu_(soln[:,i-1,:,:])*dt if mu is not None else dt
        RHS_noise = sigma_(soln[:,i-1,:,:])*dW[:,i,:,:] if sigma is not None else dW[:,i,:,:]
        
        # updating the Fourier transform for the next time step
        w = ( (1-lap_) * w + np.fft.fft2(RHS_drift + RHS_noise, axes = [-2,-1]) )/(1+lap_)
        
        # Going back to the physiscal space
        soln[:,i,:] = np.fft.ifft2(w, axes = [-2,-1]).real
        
        
    return soln

def heat_eq_2d(a, w0, T = 1, visc = 1, f = None, delta_t=1e-4, record_steps=None):

    
    #Grid size - must be power of 2
    N1, N2 = w0.shape[-2], w0.shape[-1]

    #Number of steps to final time
    steps = int(np.ceil(T/delta_t))
    if record_steps is None: record_steps = steps

    #Initial condition to Fourier space
    w_h = np.fft.fft2(w0, axes = [-2, -1])

    #Record solution every this number of steps
    record_time = steps//record_steps

    #Negative Laplacian in Fourier space
    #lap = 4*(np.pi**2)*np.array([[k1**2/a[0]**2 + k2**2/a[0]**2 for k2 in range(N2)] for k1 in range(N2)])
    # lap_ = lap.clone()
    #lap[0,0] = 0 # why was 1?
    lap = - np.array([[2*N1**2*np.cos(2*np.pi*k1/(N1*a[0])) + 2*N2**2*np.cos(2*np.pi*k2/(N2*a[1])) for k2 in range(N2)] for k1 in range(N2)]) + 2*N1**2 + 2*N2**2 
    lap[0,0] = 1
    
    #Saving solution and time
    num_of_samples = w0.shape[0] if len(w0.shape) > 2 else 1
    sol = np.zeros(shape = (num_of_samples, record_steps, N1, N2))
    sol_t = np.zeros(record_steps)
    #Record counter
    c = 0
    #Physical time
    t = 0.0
    for j in tqdm(range(steps)):
        #Stream function in Fourier space: solve Poisson equation
        
        if f is not None:
            f_h = delta_t*np.fft.fft2(f[:,j,:,:], axes = [-2, -1])
        else:
            f_h = 0
        
        w_h = ((1.0 - 0.5*delta_t*visc*lap)*w_h + f_h)/(1.0 + 0.5*delta_t*visc*lap)

        #Update real time (used only for recording)
        t += delta_t

        if (j+1) % record_time == 0:
            #Solution in physical space
            w = np.fft.ifft2(w_h, axes = [-2,-1]).real
            #Record solution and time
            sol[:,c,:,:] = w
            sol_t[c] = t
            c += 1
    if num_of_samples > 1:
        return sol, sol_t
    return sol[0], sol_t

class trees_integrator_2D():
    
    def __init__(self, visc, X, N, T, dt):
    
        self.visc = visc # viscosity in fron of laplacian
        self.X = X # edge space point of the space domain [0,X]^2
        self.N = N # number of spatial domain grid points (O_X)
        self.T = T # final time point of the time interval [0,T]
        self.dt = dt # time step

    def integrate(self, tau, done = {}, exceptions = {}, derivative = False):
        
        # TODO: Allow spatial derivatives

        #extract the trees from dictionary which are not purely polyniomials and were not already integrated
        trees = [tree for tree in tau.keys() if tree not in exceptions]
        trees = [tree for tree in trees if 'I[{}]'.format(tree) not in done and 'I[{}]'.format(tree) not in exceptions] 

        dt, N, X, M, visc = self.dt, self.N, self.X, int(np.ceil(self.T/self.dt)), self.visc

        # array of trees in Fourier space mutliplied by dt
        taus = np.fft.fft2(np.array([tau[t] for t in trees]), axes = [-2, -1])*dt 
        
        # Negative Laplacian in Fourier space
        lap = - np.array([[2*N**2*np.cos(2*np.pi*k1/(N*X)) + 2*N**2*np.cos(2*np.pi*k2/(N*X)) for k2 in range(N)] for k1 in range(N)]) + 2*N**2 + 2*N**2 
        lap[0,0] = 1
        lap_ = 0.5*dt*visc*lap

        integrated = np.zeros(shape = (len(trees), M, N, N), dtype='complex64') # planted trees in Fourier space

        # Finite difference method in Fourier space
        for i in range(M):
            
            integrated[:,i,:,:] = ((1.0 - lap_)*integrated[:,i-1,:,:] + taus[:,i,:,:])/(1.0 + lap_)
            
        integrated = np.fft.ifft2(integrated, axes = [-2, -1]).real # put planted trees to the physical space
        Jtau = {}
        for i, t in enumerate(trees): # update dictionary and return integrated taus.
            Jtau['I[{}]'.format(t)] = integrated[i]
        
        del integrated
        del taus

        return Jtau

In [2]:
def Navier_Stokes_2d(a, w0, T = 1, visc = 1, f = None, delta_t=1e-4, record_steps=1):

    #Grid size - must be power of 2
    N1, N2 = w0.shape[-2], w0.shape[-1]

    #Number of steps to final time
    steps = int(np.ceil(T/delta_t))

    #Initial vorticity to Fourier space
    w_h = np.fft.fft2(w0, axes = [-2, -1])

    #Record solution every this number of steps
    record_time = steps//record_steps

    #Negative Laplacian in Fourier space
    #lap = 4*(np.pi**2)*np.array([[k1**2/a[0]**2 + k2**2/a[0]**2 for k2 in range(N2)] for k1 in range(N1)])
    lap = - np.array([[2*(N1/a[0])**2*np.cos(2*np.pi*k1/N1) + 2*(N2/a[1])**2*np.cos(2*np.pi*k2/N2) for k2 in range(N2)] for k1 in range(N1)]) + 2*(N1/a[0])**2 + 2*(N2/a[1])**2 
    lap[0,0] = 1
    
    
    # Derivatives in the Fourier space
    diff_x = N1/a[0]*np.array([[np.cos(2*np.pi*k1/N1) + np.sin(2*np.pi*k1/N1)*1j - 1 for k2 in range(N2)] for k1 in range(N1)])
    diff_y = N2/a[1]*np.array([[np.cos(2*np.pi*k2/N2) + np.sin(2*np.pi*k2/N2)*1j - 1 for k2 in range(N2)] for k1 in range(N1)])
    
    '''# Derivatives in the Fourier space
    diff_x = -2*1j*np.pi*np.array([[k1/a[0] for k2 in range(N2)] for k1 in range(N1)])
    diff_y = -2*1j*np.pi*np.array([[k2/a[1] for k2 in range(N2)] for k1 in range(N1)])
    
    divv = -diff_x*diff_x - diff_y*diff_y
    divv[0,0] = 1'''
    
    #Saving solution and time
    if len(w0.shape) > 2:
        sol = np.zeros(shape = (w0.shape[0], record_steps, N1, N2))
    else:
        sol = np.zeros(shape = (1, record_steps, N1, N2)) 
    sol_t = np.zeros(record_steps)
    #Record counter
    c = 0
    #Physical time
    t = 0.0
    for j in tqdm(range(steps)):
        #Stream function in Fourier space: solve Poisson equation
        
        # velocity field in x and y directions. (inverse curl) 
        vx = np.fft.ifft2(diff_y*w_h, axes = [-2,-1]).real
        vy = np.fft.ifft2(-diff_x*w_h, axes = [-2,-1]).real
        
        # partials of vorticity
        wx = np.fft.ifft2(diff_x*w_h, axes = [-2,-1]).real
        wy = np.fft.ifft2(diff_y*w_h, axes = [-2,-1]).real
        
        # non-linear term
        Non_lin = - delta_t*np.fft.fft2(vx*wx + vy*wy, axes = [-2, -1])
        
        # forcing in Fourier space
        if f is not None:
            f_h = delta_t*np.fft.fft2(f[:,j,:,:], axes = [-2, -1])
        else:
            f_h = 0
        
        w_h = ((1.0 - 0.5*delta_t*visc*lap)*w_h + f_h + Non_lin)/(1.0 + 0.5*delta_t*visc*lap)

        #Update real time (used only for recording)
        t += delta_t

        if (j+1) % record_time == 0:
            #Solution in physical space
            w = np.fft.ifft2(w_h, axes = [-2,-1]).real
            #Record solution and time
            sol[:,c,:,:] = w
            sol_t[c] = t
            c += 1
    if len(w0.shape) > 2:
        return sol, sol_t
    return sol[0], sol_t